In [2]:
%use dataframe, lets-plot

In [3]:
@DataSchema
interface Player {

    @ColumnName("Player")
    val player: String?

    @ColumnName("KDA")
    val kda: Double?

    @ColumnName("CSM")
    val csm: Double?

    @ColumnName("KP%")
    val killParticipationRate: String?

    @ColumnName("DPM")
    val dpm: Int
}

In [15]:
import org.jetbrains.letsPlot.intern.Plot

val midLinePlayersDataFrame = DataFrame.readCSV("resources/LCK_summer_2023_mid_line_players.csv").cast<Player>()

val comparePlayers = listOf("Faker", "Chovy", "Bdd")
val comparePlayersInfo = comparePlayers.map { player ->
    midLinePlayersDataFrame.first { it.player?.equals(player, ignoreCase = true) == true }
}

val averageKda = midLinePlayersDataFrame.map { kda ?: 0.0 }.average()
val averageCsm = midLinePlayersDataFrame.map { csm ?: 0.0 }.average()
val averageKpr = midLinePlayersDataFrame.map { killParticipationRate?.replace("%", "")?.toDoubleOrNull()?.roundToInt() ?: 0 }.average()

val kdaPlotData = mapOf(
    "player" to comparePlayers + "LCK 평균",
    "kda" to comparePlayersInfo.map { it.kda } + averageKda
)
val csmPlotData = mapOf(
    "player" to comparePlayers + "LCK 평균",
    "csm" to comparePlayersInfo.map { it.csm } + averageCsm
)
val kprPlotData = mapOf(
    "player" to comparePlayers + "LCK 평균",
    "kpr" to comparePlayersInfo.map { it.killParticipationRate?.replace("%", "")?.toDoubleOrNull()?.roundToInt() ?: 0 } + averageKpr
)

fun createPlot(data: Map<*, *>, y: String): Plot {
    return letsPlot(data) +
            geomBar(stat = Stat.identity) { x = "player"; this.y = y }
}

gggrid(
    plots = listOf(
        createPlot(kdaPlotData, "kda"),
        createPlot(csmPlotData, "csm"),
        createPlot(kprPlotData, "kpr")
    )
)
